# TRAITEMENT DES IMAGES

ici nous allons montrer comment nous traitons nos images

# Voici le code qui renome les images

le code a des variable au debut qui permet de changer rapidement les nom de dossier et nom de fichier pour modifier les noms

In [3]:
# %load renamefile.py
import os

basket = 'Balle_de_basketball'
football = 'Balle_de_football'
baseball = "Balle_de_baseball"

nameBasketball = "basketball_"
nameFootball = "football_"
nameBaseball = "baseball_"

data1 = "dataset"
data2 = "dataset_copy"
data3 = "dataset_Same_Size"

# Chemin du dossier contenant les images à renommer
dirname = os.getcwd()
source_dir = os.path.join(dirname, "..", data3, basket)

# Liste des fichiers dans le dossier
files = os.listdir(source_dir)

# Liste des numéros de fichiers existants
existing_nums = []
for file in files:
    if file.endswith(".jpg") or file.endswith(".png") or file.endswith('jpeg'):
        num = int(file.replace('.jpg','').replace('.png','').replace('.jpeg','').split('_')[-1])
        existing_nums.append(num)

# Compteur pour numéroter les images
count = 1

# Pour chaque fichier dans le dossier
for file in files:
    # Vérification que le fichier est bien une image
    if file.endswith(".jpg") or file.endswith(".png") or file.endswith('jpeg'):
        # Chemin complet du fichier
        source_path = os.path.join(source_dir, file)
        
        # Saute les numéros de fichiers existants
        while count in existing_nums:
            count += 1
        
        # Nouveau nom de fichier avec numéro séquentiel
        new_filename = f"{nameBasketball}{count}.jpg"
        # Chemin complet du nouveau fichier
        dest_path = os.path.join(source_dir, new_filename)
        # Renommage du fichier
        os.rename(source_path, dest_path)
        # Incrément du compteur
        count += 1

print("Les images ont été renommées avec succès !")

ValueError: invalid literal for int() with base 10: 'ballon-basket-3x3-molten'

# Le changement de tailles des images

ici on definie encore des constants pour faire le changement de taille par dossier d'image.

In [ ]:
# %load resize_image.py
from PIL import Image
import os

basket = "Balle_de_basketball"
football = "Balle_de_football"
baseball = "Balle_de_baseball"

def resize_image(image_path, output_path, size=(200, 200)):
    with Image.open(image_path) as img:
        if img.mode == "CMYK":
            img = img.convert("RGB")
        img_resized = img.resize(size)
        img_resized.save(output_path)
        # Fermeture de l'image redimensionnée
        img_resized.close()

def know_all_size(directory):
    # Liste de tous les fichiers dans le dossier
    files = os.listdir(directory)
    # Pour chaque fichier dans le dossier
    for file in files:
        # Vérification que le fichier est bien une image
        if file.endswith(".jpg"):
            # Chemin complet du fichier
            image_path = os.path.join(directory, file)
            # Ouverture de l'image et affichage de sa taille en pixels
            with Image.open(image_path) as img:
                print(f"La taille de l'image {file} est: {img.size}")
            # Fermeture de l'image en dehors du bloc "with"
            img.close()

def rezise_all_image(directory, x, y):
    # Liste de tous les fichiers dans le dossier
    files = os.listdir(directory)
    # Pour chaque fichier dans le dossier
    for file in files:
        # Vérification que le fichier est bien une image
        if file.endswith(".jpg"):
            # Chemin complet du fichier
            image_path = os.path.join(directory, file)
            # Ouverture de l'image et affichage de sa taille en pixels
            with Image.open(image_path) as img:
                img_resized = img.resize((x, y))
                img_resized.save(image_path)
                # Fermeture de l'image redimensionnée
                img_resized.close()

dirname = os.path.abspath(os.path.dirname(__file__))
images_folder = os.path.join(dirname, "..", "dataset", football)
output_folder = os.path.join(dirname, "..", "dataset_Same_Size", football)
new_size = (25, 25) # Taille de la nouvelle image

# Parcourez chaque image dans le dossier images_folder
for filename in os.listdir(images_folder):
    # Obtenez le chemin complet de l'image originale et de la nouvelle image redimensionnée
    image_path = os.path.join(images_folder, filename)
    output_path = os.path.join(output_folder, filename)
    # Redimensionner l'image
    resize_image(image_path, output_path, size=new_size)

know_all_size(output_folder)

#rezise_all_image(output_folder, new_size=(25, 25))

# Traitement des images pour l'apprentissage et les prediction

la fonction expected_image va faire l'étiquetage des donnée

allcolors lui va ouvrir les images et récuperer les pixels des images en les stockant dans des tableaux

In [ ]:
# %load create_listTest.py
import os 
from PIL import Image
import numpy as np
import ctypes
import sys

def expected_image(directory):
    expected = []
    for filename in os.listdir(directory):
        if filename.startswith("basketball_"):
            expected.append([0,0,1])
        elif filename.startswith("football_"):
            expected.append([0,1,0])
        elif filename.startswith("baseball_"):
            expected.append([1,0,0])
        else:
            print(f"Le fichier {filename} n'a pas été étiqueté car son nom ne commence pas par basketball_, football_ ou baseball_.")
    
    # Convertir les étiquettes en one-hot encoding
    expected = np.array(expected, dtype=np.double)
    
    return expected


def allcolors(directory):
    all_pixels = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            image_path = os.path.join(directory, filename)
            image = Image.open(image_path)
            if image.mode != 'RGB':
                print(f"L'image {filename} n'est pas en format RGB (mode {image.mode})")
                image = image.convert('RGB')
            pixels = []
            for pixel in image.getdata():
                r, g, b = pixel
                pixels.extend([r / 255.0, g / 255.0, b / 255.0])
            all_pixels.append(pixels)
    if all_pixels:
        # Convertir les pixels en tableau numpy
        all_pixels = np.array(all_pixels, dtype=np.double)
        return all_pixels.reshape((all_pixels.shape[0], -1))
    else:
        print("Aucune image n'a été trouvée dans le dossier.")